In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
dataset = pd.read_csv("./NYPD_Shooting_Incident_Data__Year_To_Date_.csv")


In [16]:
dataset.drop(columns=["INCIDENT_KEY","OCCUR_DATE", "OCCUR_TIME", "JURISDICTION_CODE", "LOCATION_DESC"])

,BORO,LOC_OF_OCCUR_DESC,PRECINCT,LOC_CLASSFCTN_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,...,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column,Borough Boundaries,City Council Districts,Police Precincts,Zip Codes,Community Districts
0,BRONX,OUTSIDE,49,STREET,Y,18-24,M,BLACK HISPANIC,18-24,F,...,1020669,255027,40.866590,-73.868328,POINT (-73.86832754 40.86658962),5.0,22.0,32.0,11605.0,59.0
1,BRONX,OUTSIDE,52,STREET,N,25-44,M,WHITE HISPANIC,25-44,M,...,1008387,253290,40.861864,-73.912739,POINT (-73.91273918 40.86186421),5.0,29.0,34.0,11606.0,24.0
2,BRONX,OUTSIDE,44,VEHICLE,N,(null),(null),(null),25-44,M,...,1008878,244480,40.837683,-73.910994,POINT (-73.910994 40.837683),5.0,42.0,27.0,10935.0,50.0
3,QUEENS,OUTSIDE,113,HOUSING,N,(null),(null),(null),18-24,M,...,1044794,188802,40.684689,-73.781699,POINT (-73.781699 40.684689),3.0,46.0,71.0,24669.0,41.0
4,BROOKLYN,INSIDE,77,DWELLING,N,(null),(null),(null),25-44,F,...,1002013,185752,40.676508,-73.935960,POINT (-73.93596 40.676508),2.0,49.0,49.0,17615.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,BRONX,OUTSIDE,48,STREET,N,(null),(null),(null),18-24,M,...,1011526,247828,40.846864,-73.901413,POINT (-73.90141321 40.84686352),5.0,22.0,29.0,10935.0,6.0
1712,MANHATTAN,OUTSIDE,30,STREET,N,25-44,F,WHITE HISPANIC,25-44,M,...,997458,240485,40.826743,-73.952273,POINT (-73.952273 40.826743),4.0,23.0,19.0,12428.0,37.0
1713,MANHATTAN,OUTSIDE,32,STREET,N,18-24,M,BLACK,25-44,M,...,1000999,234464,40.810209,-73.939496,POINT (-73.9394955 40.81020941),4.0,36.0,20.0,13095.0,18.0
1714,BRONX,INSIDE,46,HOUSING,N,25-44,M,BLACK,25-44,M,...,1012980,251028,40.855644,-73.896141,POINT (-73.896141 40.855644),5.0,22.0,29.0,10935.0,6.0
